In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import reshape
import pandas

In [2]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_ids = test["PassengerId"]

In [3]:
def clean(data):
    data = data.drop(["Ticket", "Cabin", "PassengerId"], axis=1)
    cols = ["SibSp", "Parch", "Age", "Fare"]
    
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
        
    data.Embarked.fillna("U", inplace=True)
    
    def clean_name(value):
        if ("Mr." in value): value = "mr"
        if ("Miss." in value): value = "miss"
        if ("Ms." in value): value = "ms"
        if ("Mrs." in value): value = "mrs"
        if ("Don." in value) or ("Dona." in value): value = "Don/Dona"
        if ("Master." in value): value = "master"
        if ("Col." in value): value = "col"
        if ("Rev." in value): value = "rev"
        if ("Dr." in value): value = "Dr"
        
        return value
    
    data["Name"] = [clean_name(i) for i in data["Name"]]    
    return data


data = clean(data)
test = clean(test)

In [4]:
le = preprocessing.LabelEncoder()

cols_a = ["Sex", "Embarked", "Name"]

for col in cols_a:
    data[col] = le.fit_transform(data[col])
    test[col] = le.fit_transform(test[col])

def scale(data):
    scaler = preprocessing.MinMaxScaler()
    scaler.fit(data)
    data = scaler.transform(data)
    data = pandas.core.api.DataFrame(data, columns=["Survived", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"])    
    return data
    
data = scale(data)  

y = data["Survived"]
X = data.drop("Survived", axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:

classifier  = KNeighborsClassifier(n_neighbors = 9)
classifier.fit(X_train, y_train)

score_predict = classifier.predict(X_val)
accuracy_score(y_val, score_predict)

0.8156424581005587

In [6]:
final_predict = classifier.predict(test)
final_predict = [int(i) for i in final_predict]


df = pd.DataFrame({"PassengerId":test_ids.values,
                  "Survived": final_predict})
df.to_csv("Submission3.5.csv", index=False)

In [7]:
# 0.7988826815642458 w "names"
# 0.7821229050279329 no "name"
# best k = 9 (score = 0.81)